#Aim

Given a large set of sequences or graphs with ordered vertices find small vertex ordered subsequences that are most discriminative for the set.

Steps:
- devise a negative set
- learn a discriminative model
- annotate importance on vertices
- extract max subarrays 
- cluster them 
 - use fast EDeN string kernel 
 - custering algorithm
 
Output: 
1. all sequence motives in each cluster
2. all initial sequences with motif location (begin,end) and cluster id (build regex from all seqs in cluster and run a find iterator)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#code for making artificial dataset
import random
def random_string(length,alphabet_list):
    rand_str = ''.join(random.choice(alphabet_list) for i in range(length))
    return rand_str

def perturb(seed,alphabet_list,p=0.5):
    seq=''
    for c in seed:
        if random.random() < p: c = random.choice(alphabet_list)
        seq += c
    return seq

def make_artificial_dataset(alphabet='ACGU', motives=None, motif_length=6, sequence_length=100, n_sequences=1000, n_motives=2, p=0.2):
    alphabet_list=[c for c in alphabet]
    
    if motives is None:
        motives=[]
        for i in range(n_motives):
            motives.append(random_string(motif_length,alphabet_list))
    else:
        motif_length = len(motives[0])
        n_motives = len(motives)
        
    flanking_length = (sequence_length - motif_length ) / 2
    n_seq_per_motif = n_sequences / n_motives

    counter=0
    seqs=[]
    for i in range(n_seq_per_motif):
        for j in range(n_motives):
            left_flanking = random_string(flanking_length,alphabet_list)
            right_flanking = random_string(flanking_length,alphabet_list)
            noisy_motif = perturb(motives[j],alphabet_list,p)
            seq = left_flanking + noisy_motif + right_flanking
            seqs.append(('>ID%d'%counter,seq))
            counter += 1
    return motives, seqs

In [3]:
from eden.motif import SequenceMotif
help(SequenceMotif)

Help on class SequenceMotif in module eden.motif:

class SequenceMotif(__builtin__.object)
 |  Methods defined here:
 |  
 |  __init__(self, min_subarray_size=7, max_subarray_size=10, min_motif_count=1, min_cluster_size=1, training_size=None, negative_ratio=2, shuffle_order=2, n_iter_search=1, complexity=4, nbits=20, clustering_algorithm=None, n_jobs=4, n_blocks=8, block_size=None, pre_processor_n_jobs=4, pre_processor_n_blocks=8, pre_processor_block_size=None, random_state=1)
 |  
 |  fit(self, seqs, neg_seqs=None)
 |      Builds a discriminative estimator.
 |      Identifies the maximal subarrays in the data.
 |      Clusters them with the clustering algorithm provided in the initialization phase.
 |      For each cluster builds a fast sequence search model (Aho Corasick data structure).
 |  
 |  fit_predict(self, seqs, return_list=False)
 |  
 |  fit_transform(self, seqs, return_match=False)
 |  
 |  load(self, obj)
 |  
 |  predict(self, seqs, return_list=False)
 |      Returns for

#Experimental Setup

In [4]:
#setup parameters
alphabet='ACGU'
motives=['AAAAAAAAAA','CCCCCCCCCC','GGGGGGGGGG','UUUUUUUUUU']
sequence_length=100
n_sequences=400
p=0.3

#make dataset
motives, seqs = make_artificial_dataset(alphabet=alphabet,motives=motives,sequence_length=sequence_length,n_sequences=n_sequences,p=p)

#display
print 'Motives and sample of their perturbed variants:'
alphabet_list=[c for c in alphabet]
for motif in motives: 
    print
    print motif,
    for i in range(9):
        print perturb(motif,alphabet_list,p=p),

Motives and sample of their perturbed variants:

AAAAAAAAAA GAAAUAAAUA AAAAAAAAAA AAAAGUAUAA ACAAAAAAAU CCAUUACAAA UUAAAUAAAA AAAAAAAAAA AACACUUAAA AUAACAACAA
CCCCCCCCCC CGCUCCCCUC CCCCCCCCCC CGCCCGCCCC CCCCCCCCCC ACGCCCCCUC CCCGCCCUCA CCUCCCCCCC CCCCCACCCC CCGCACCCUA
GGGGGGGGGG GGGGGGAGGG GCGGGCGGGG GGGGGCGGCG GGGAGGGGAG GGAGGCUUGG GGGAGAGGGG GUGAGAGGGG GAGGGCGGGG GGGGGGGGGA
UUUUUUUUUU UUUUUUGUUU GAUUUUUUCA UUUUUUUUUU UUUCUUUUUU UUUAUUUUUU UUUUGUCUCU UGUUUUUUUU AUUAUUUUCU UUUUUUCUUU


In [5]:
#save to file
fname='artificial_motif_search_dataset.fa'
with open(fname,'w') as f:
    for header,seq in seqs: 
        f.write(header+"\n")
        f.write(seq+"\n")

#save explicit negative sequences
from eden.modifier.seq import seq_to_seq, shuffle_modifier
neg_seqs = list(seq_to_seq(seqs, modifier=shuffle_modifier, times=2, order=2))
fname='artificial_motif_search_dataset_negatives.fa'
with open(fname,'w') as f:
    for header,seq in neg_seqs: 
        f.write(header+"\n")
        f.write(seq+"\n")

In [6]:
from eden.util import configure_logging
import logging
configure_logging(logging.getLogger(),verbosity=2)

In [7]:
%%time
from sklearn.cluster import Birch
ca = Birch(threshold=0.1, n_clusters=4, branching_factor=50)

from eden.motif import SequenceMotif
seqmot = SequenceMotif(complexity=2, nbits=14, clustering_algorithm=ca, min_motif_count=2, n_jobs=2)
seqmot.fit(seqs, neg_seqs)
seqmot.save('seqmot')

Positive data: Instances: 400 ; Features: 16385 with an avg of 433 features per instance
Negative data: Instances: 800 ; Features: 16385 with an avg of 439 features per instance
Elapsed time: 14.4 secs
model induction: 400 positive instances 15 secs
motives extraction: 271 motives 5 secs
motives clustering: 4 clusters 0 secs
after filtering: 30 motives 4 clusters 0 secs
motif model construction: 0 secs
CPU times: user 11.7 s, sys: 1.52 s, total: 13.2 s
Wall time: 21.6 s


In [8]:
for cluster_id in seqmot.motives_db:
    print cluster_id
    for count, motif in sorted(seqmot.motives_db[cluster_id], reverse=True):
        print motif, count

0
AAAAAAAAAA 9
AAAAAAAAA 4
AUAAAAAAAA 2
AGAAAAAAA 2
ACAAAAAAAA 2
ACAAAAAAA 2
AAAUAAAAA 2
1
CCCCCCCCCC 7
CCCCCCCCC 5
CGCCCCCCC 2
CCCCUCCCCC 2
CCCCCUCCU 2
CCCCCGCCCC 2
CCCCCCCCGC 2
CCCCCCCCCA 2
CCCCCACCC 2
CACCCCACCC 2
2
UUUUUUUUUU 5
UUUUUUUUU 3
UUUUUUUCU 2
UUUCUUUUUC 2
UUUCUUUUU 2
UCUUUUUUUU 2
UCUUUUUUU 2
GUUUUUUUUU 2
3
GGGGGGGGGG 10
GGGGGGGCG 4
GGGGGGGGG 2
GGGGGCGGGG 2
GGGCGGGGGG 2


In [9]:
from eden.motif import SequenceMotif
seqmot2 = SequenceMotif()
seqmot2.load('seqmot')

predictions=seqmot2.predict(seqs, return_list=True)
for p in predictions: print p

[]
[1]
[3]
[]
[]
[1]
[]
[]
[]
[]
[3]
[]
[0]
[]
[]
[2]
[]
[]
[]
[]
[]
[]
[]
[2]
[]
[1]
[]
[2]
[0]
[]
[]
[2]
[]
[]
[]
[2]
[0]
[]
[]
[]
[]
[]
[]
[2]
[0]
[]
[]
[2]
[]
[1]
[3]
[2]
[]
[]
[]
[]
[0]
[1]
[3]
[]
[]
[]
[]
[]
[]
[1]
[]
[]
[]
[]
[]
[]
[0]
[]
[]
[]
[]
[]
[]
[]
[0]
[]
[]
[2]
[]
[1]
[]
[2]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[1]
[]
[]
[]
[]
[]
[2]
[]
[1]
[3]
[]
[]
[1]
[3]
[]
[]
[]
[3]
[]
[]
[]
[]
[]
[]
[1]
[3]
[]
[0]
[]
[]
[]
[]
[]
[]
[]
[0]
[1]
[]
[]
[0]
[]
[]
[]
[]
[1]
[]
[2]
[]
[]
[]
[]
[]
[1]
[]
[2]
[]
[]
[]
[2]
[]
[]
[3]
[]
[0]
[]
[]
[]
[0]
[1]
[3]
[]
[]
[1]
[]
[]
[]
[]
[]
[2]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[1]
[3]
[]
[]
[1]
[]
[]
[]
[1]
[3]
[]
[0]
[]
[]
[2]
[]
[1]
[3]
[]
[0]
[]
[3]
[2]
[0]
[]
[]
[]
[]
[]
[]
[2]
[]
[]
[]
[]
[0]
[]
[]
[]
[]
[]
[3]
[]
[]
[1]
[]
[]
[]
[]
[]
[]
[0]
[]
[]
[2]
[]
[]
[]
[]
[]
[]
[3]
[]
[0]
[]
[]
[]
[]
[]
[]
[]
[0]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[3]
[2]
[]
[]
[3]
[2]
[0]
[]
[]
[2]
[]
[]
[]
[2]
[0]
[]
[]
[2]
[0]
[]
[3]
[2]
[0]
[1]
[3]
[

In [10]:
predictions=seqmot2.predict(seqs, return_list=False)
for p in predictions: print p

0
1
1
0
0
1
0
0
0
0
1
0
1
0
0
1
0
0
0
0
0
0
0
1
0
1
0
1
1
0
0
1
0
0
0
1
1
0
0
0
0
0
0
1
1
0
0
1
0
1
1
1
0
0
0
0
1
1
1
0
0
0
0
0
0
1
0
0
0
0
0
0
1
0
0
0
0
0
0
0
1
0
0
1
0
1
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
1
0
1
1
0
0
1
1
0
0
0
1
0
0
0
0
0
0
1
1
0
1
0
0
0
0
0
0
0
1
1
0
0
1
0
0
0
0
1
0
1
0
0
0
0
0
1
0
1
0
0
0
1
0
0
1
0
1
0
0
0
1
1
1
0
0
1
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
1
1
0
0
1
0
0
0
1
1
0
1
0
0
1
0
1
1
0
1
0
1
1
1
0
0
0
0
0
0
1
0
0
0
0
1
0
0
0
0
0
1
0
0
1
0
0
0
0
0
0
1
0
0
1
0
0
0
0
0
0
1
0
1
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
1
1
0
0
1
1
1
0
0
1
0
0
0
1
1
0
0
1
1
0
1
1
1
1
1
0
0
1
0
0
0
0
0
0
1
1
0
0
1
1
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
1
0
0
0
0
1
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
1
1
1
1
0
0
1
0
0
1
0
0
1
0
1
0
0
0
1
1
0
0
0
0
1
0
1
1
1


In [11]:
predictions=seqmot2.transform(seqs, return_match=True)
for p in predictions: print p

[[], [], [], []]
[[], [(45, 54), (87, 96)], [], []]
[[], [], [], [(46, 55)]]
[[], [], [], []]
[[], [], [], []]
[[], [(44, 53), (46, 55), (46, 56)], [], []]
[[], [], [], []]
[[], [], [], []]
[[], [], [], []]
[[], [], [], []]
[[], [], [], [(46, 56)]]
[[], [], [], []]
[[(46, 55)], [], [], []]
[[], [], [], []]
[[], [], [], []]
[[], [], [(45, 54)], []]
[[], [], [], []]
[[], [], [], []]
[[], [], [], []]
[[], [], [], []]
[[], [], [], []]
[[], [], [], []]
[[], [], [], []]
[[], [], [(43, 52), (47, 56)], []]
[[], [], [], []]
[[], [(45, 55)], [], []]
[[], [], [], []]
[[], [], [(42, 51)], []]
[[(47, 56)], [], [], []]
[[], [], [], []]
[[], [], [], []]
[[], [], [(43, 53), (44, 53)], []]
[[], [], [], []]
[[], [], [], []]
[[], [], [], []]
[[], [], [(43, 53), (44, 53), (44, 54), (45, 54)], []]
[[(45, 54)], [], [], []]
[[], [], [], []]
[[], [], [], []]
[[], [], [], []]
[[], [], [], []]
[[], [], [], []]
[[], [], [], []]
[[], [], [(46, 55)], []]
[[(45, 54), (45, 55), (46, 55)], [], [], []]
[[], [], [], []

In [12]:
predictions=seqmot2.transform(seqs, return_match=False)
for p in predictions: print p

[0, 0, 0, 0]
[0, 1, 0, 0]
[0, 0, 0, 1]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 1, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 1]
[0, 0, 0, 0]
[1, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 1, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 1, 0]
[0, 0, 0, 0]
[0, 1, 0, 0]
[0, 0, 0, 0]
[0, 0, 1, 0]
[1, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 1, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 1, 0]
[1, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 1, 0]
[1, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 1, 0]
[0, 0, 0, 0]
[0, 1, 0, 0]
[0, 0, 0, 1]
[0, 0, 1, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[1, 0, 0, 0]
[0, 1, 0, 0]
[0, 0, 0, 1]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 1, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[1, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]

In [13]:
%%time
from sklearn.cluster import MiniBatchKMeans
ca = MiniBatchKMeans(n_clusters=4)

from eden.motif import SequenceMotif
seqmot = SequenceMotif(clustering_algorithm=ca, min_motif_count=2)
seqmot.fit(seqs)

for cluster_id in seqmot.motives_db:
    print cluster_id
    for count, motif in sorted(seqmot.motives_db[cluster_id], reverse=True):
        print motif, count

Positive data: Instances: 400 ; Features: 1048577 with an avg of 1734 features per instance
Negative data: Instances: 800 ; Features: 1048577 with an avg of 1751 features per instance
Elapsed time: 21.4 secs
model induction: 400 positive instances 22 secs
motives extraction: 286 motives 6 secs
motives clustering: 4 clusters 0 secs
after filtering: 29 motives 4 clusters 0 secs
motif model construction: 0 secs
0
CCCCCCCCCC 7
CCCCCGCCCC 3
CCCCCCCCC 3
CCCCUCCCCC 2
CCCCUCCCC 2
CCCCCUCCCC 2
CCCCCCCCCG 2
CCCCCCCCCA 2
CCCCCACCCC 2
CCCCACCCA 2
1
AAAAAAAAAA 5
AAAAAAAAA 5
AAAAAAAAU 3
GAAAAAAAU 2
AUAAAAAAAA 2
AAAUAAAAAA 2
AAAAAAGAAA 2
AAAAAAAAAC 2
2
UUUUUUUUUU 5
CUUUUUUUUC 4
UUUCUUUUUC 3
UUUUUUUUUG 2
UUUUUUUUUC 2
UCUUUUUUU 2
CUUUUUUUUU 2
3
GGGGGGGGGG 11
GGGGGGGCGA 2
GGGGAGGGCG 2
GGGCGGGGGG 2
CPU times: user 14.4 s, sys: 2.26 s, total: 16.6 s
Wall time: 30 s


In [14]:
%%time
from sklearn.cluster import DBSCAN
ca = DBSCAN(eps=0.1, min_samples=3)

from eden.motif import SequenceMotif
seqmot = SequenceMotif(clustering_algorithm=ca, min_motif_count=2)
seqmot.fit(seqs)

for cluster_id in seqmot.motives_db:
    print cluster_id
    for count, motif in sorted(seqmot.motives_db[cluster_id], reverse=True):
        print motif, count

Positive data: Instances: 400 ; Features: 1048577 with an avg of 1734 features per instance
Negative data: Instances: 800 ; Features: 1048577 with an avg of 1751 features per instance
Elapsed time: 21.3 secs
model induction: 400 positive instances 22 secs
motives extraction: 276 motives 6 secs
motives clustering: 13 clusters 0 secs
after filtering: 12 motives 12 clusters 0 secs
motif model construction: 0 secs
0
CUUUUUUUUC 4
1
CCCCCGCCCC 3
2
CUUUUUUUA 3
3
UUUUUUUUUU 5
4
AAAAAAAAAA 4
5
UUUCUUUUUC 3
6
CCCCCCCCCC 5
7
CCCCCCCCC 4
8
GGGGGGGGG 3
9
GGGGGGGGGG 10
10
CUUUUUUUUU 3
11
AAAAAAAAA 5
CPU times: user 14 s, sys: 3.15 s, total: 17.1 s
Wall time: 29.4 s
